<h2>
  Preprocesamiento (al menos una valida, otros dos por ver los resultados, si no se aplica justifique porque), Balanceo de datos
</h2>

In [8]:
# Importar librerías
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

from google.colab import drive
drive.mount('/content/drive')

#subimo el archivoo

data = pd.read_csv('/content/drive/My Drive/datasets/laptop_prices.csv')

# Crear DataFrame
df = pd.DataFrame(data)

# Mostrar el DataFrame original
print("DataFrame original:")
print(df)

# 1. **Limpieza de datos**

# a) Eliminar valores faltantes
df_cleaned = df.dropna()  # Eliminar filas con valores faltantes

# b) Eliminar duplicados
df_cleaned = df_cleaned.drop_duplicates()  # Eliminar filas duplicadas

# Mostrar el DataFrame limpio
print("\nDataFrame después de eliminar NaN y duplicados:")
print(df_cleaned)

# 2. **Conversión de tipos de datos**

# Convertir variables categóricas a numéricas usando LabelEncoder
label_encoder = LabelEncoder()

# Convertir columnas categóricas a numéricas
df['Company'] = label_encoder.fit_transform(df['Company'])
df['Product'] = label_encoder.fit_transform(df['Product'])
df['TypeName'] = label_encoder.fit_transform(df['TypeName'])
df['OS'] = label_encoder.fit_transform(df['OS'])
df['Screen'] = label_encoder.fit_transform(df['Screen'])
df['Touchscreen'] = label_encoder.fit_transform(df['Touchscreen'])
df['IPSpanel'] = label_encoder.fit_transform(df['IPSpanel'])
df['RetinaDisplay'] = label_encoder.fit_transform(df['RetinaDisplay'])
df['CPU_company'] = label_encoder.fit_transform(df['CPU_company'])
df['CPU_model'] = label_encoder.fit_transform(df['CPU_model'])
df['PrimaryStorageType'] = label_encoder.fit_transform(df['PrimaryStorageType'])
df['SecondaryStorageType'] = label_encoder.fit_transform(df['SecondaryStorageType'])
df['GPU_company'] = label_encoder.fit_transform(df['GPU_company'])
df['GPU_model'] = label_encoder.fit_transform(df['GPU_model'])

# Mostrar DataFrame con variables categóricas convertidas
print("\nDataFrame con variables categóricas convertidas a numéricas:")
print(df)

# 3. **Normalización de datos numéricos**

# Normalizar las columnas numéricas
scaler = StandardScaler()

# Normalizamos solo las columnas numéricas (en este caso, 'Inches', 'Ram', 'Weight', 'Price_euros', 'ScreenW', 'ScreenH', 'CPU_freq', 'PrimaryStorage', 'SecondaryStorage')
df[['Inches', 'Ram', 'Weight', 'Price_euros', 'ScreenW', 'ScreenH', 'CPU_freq', 'PrimaryStorage', 'SecondaryStorage']] = scaler.fit_transform(
    df[['Inches', 'Ram', 'Weight', 'Price_euros', 'ScreenW', 'ScreenH', 'CPU_freq', 'PrimaryStorage', 'SecondaryStorage']])

# Mostrar el DataFrame después de la normalización
print("\nDataFrame después de normalizar las columnas numéricas:")
print(df)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame original:
     Company                              Product            TypeName  Inches  \
0      Apple                          MacBook Pro           Ultrabook    13.3   
1      Apple                          Macbook Air           Ultrabook    13.3   
2         HP                               250 G6            Notebook    15.6   
3      Apple                          MacBook Pro           Ultrabook    15.4   
4      Apple                          MacBook Pro           Ultrabook    13.3   
...      ...                                  ...                 ...     ...   
1270  Lenovo                       Yoga 500-14ISK  2 in 1 Convertible    14.0   
1271  Lenovo                       Yoga 900-13ISK  2 in 1 Convertible    13.3   
1272  Lenovo                   IdeaPad 100S-14IBR            Notebook    14.0   
1273      HP  15-AC110nv (i7-6500U/6GB/1T

 **Balanceo de datos**
 El balanceo es importante cuando las clases están desbalanceadas, es decir, si algunas clases tienen muchos más ejemplos que otras.

 SMOTE es una técnica que se utiliza para aumentar el número de muestras de las clases minoritarias generando muestras sintéticas (no copias exactas) en lugar de simplemente duplicar los ejemplos de las clases minoritarias

In [9]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Aplicamos SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)

# Aplicar SMOTE
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Ver distribución original de clases en y_train
print("\nDistribución original de clases en y_train:")
print(Counter(y_train))

# Ver distribución de clases después de aplicar SMOTE
print("\nDistribución balanceada de clases en y_resampled:")
print(Counter(y_resampled))

# Mostrar una parte de X_resampled y y_resampled para verificar el balanceo
print("\nPrimeras filas de X_resampled (después de SMOTE):")
print(X_resampled[:5])

print("\nPrimeras filas de y_resampled (después de SMOTE):")
print(y_resampled[:5])



Distribución original de clases en y_train:
Counter({3: 568, 1: 161, 4: 160, 0: 89, 5: 23, 2: 19})

Distribución balanceada de clases en y_resampled:
Counter({0: 568, 1: 568, 3: 568, 4: 568, 2: 568, 5: 568})

Primeras filas de X_resampled (después de SMOTE):
   Company  Product    Inches       Ram  OS    Weight  Price_euros  Screen  \
0        0       79 -1.205746 -0.086499   5 -0.658549    -0.493890       1   
1       14       85 -0.715862  1.483418   5 -0.135328     2.518326       1   
2       10      581  0.403873 -0.871458   5  0.088909    -0.382552       1   
3        4      239  0.403873 -0.086499   2  0.238401    -0.763863       1   
4        7       50  0.403873 -0.871458   5 -0.269871    -0.950736       1   

    ScreenW   ScreenH  ...  RetinaDisplay  CPU_company  CPU_freq  CPU_model  \
0  0.040466  0.021481  ...              0            1  0.391185         55   
1  0.040466  0.021481  ...              0            1  0.986839         66   
2  0.040466  0.021481  ...        

**Primera ejecución: Confiabilidad, matriz de confusión**

In [10]:
 # 4. **Preparar las variables de entrada (X) y objetivo (y)**

X = df.drop(columns=['TypeName'])  # Variables de entrada
y = df['TypeName']  # Variable objetivo

# Ver la distribución de las clases en la variable objetivo
# print("\nDistribución de las clases de 'TypeName':")
# print(Counter(y))

# Dividir en conjunto de entrenamiento y conjunto de prueba (opcional)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar los primeros registros del conjunto de entrenamiento
print("\nConjunto de entrenamiento (X_train y y_train):")
print(X_train.head())




Conjunto de entrenamiento (X_train y y_train):
      Company  Product    Inches       Ram  OS    Weight  Price_euros  Screen  \
413         0       79 -1.205746 -0.086499   5 -0.658549    -0.493890       1   
778        14       85 -0.715862  1.483418   5 -0.135328     2.518326       1   
1107       10      581  0.403873 -0.871458   5  0.088909    -0.382552       1   
96          4      239  0.403873 -0.086499   2  0.238401    -0.763863       1   
309         7       50  0.403873 -0.871458   5 -0.269871    -0.950736       1   

       ScreenW   ScreenH  ...  RetinaDisplay  CPU_company  CPU_freq  \
413   0.040466  0.021481  ...              0            1  0.391185   
778   0.040466  0.021481  ...              0            1  0.986839   
1107  0.040466  0.021481  ...              0            1 -0.005918   
96    0.040466  0.021481  ...              0            1  0.788288   
309   0.040466  0.021481  ...              0            1 -0.601572   

      CPU_model  PrimaryStorage  Secon

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Inicializar el clasificador
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión (confiabilidad)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo (accuracy): {accuracy}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)


Precisión del modelo (accuracy): 0.8862745098039215
Matriz de Confusión:
[[ 20   1   0   4   3   0]
 [  0  43   0   1   0   0]
 [  0   0   3   0   1   0]
 [  1   2   0 131   5   0]
 [  1   0   0   8  25   0]
 [  0   1   0   1   0   4]]


In [12]:
# Inicializar lista para almacenar la confiabilidad (precisión)
confiabilidades = []

# Ejecutar 100 splits para obtener la mediana de la confiabilidad
for _ in range(100):
    # División aleatoria 80/20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Realizar predicciones
    y_pred = model.predict(X_test)

    # Calcular la precisión (accuracy)
    accuracy = accuracy_score(y_test, y_pred)
    confiabilidades.append(accuracy)

# Calcular la mediana de la confiabilidad
mediana_confiabilidad = np.median(confiabilidades)
print(f'Mediana de la confiabilidad (precisión) en 100 splits: {mediana_confiabilidad}')


Mediana de la confiabilidad (precisión) en 100 splits: 0.9


In [13]:
# División 50/50 para entrenamiento y prueba
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(X, y, test_size=0.5, random_state=42)

# Entrenar el modelo en el nuevo conjunto 50/50
model.fit(X_train_50, y_train_50)

# Realizar predicciones
y_pred_50 = model.predict(X_test_50)

# Calcular la precisión para esta división 50/50
accuracy_50 = accuracy_score(y_test_50, y_pred_50)
print(f'Precisión del modelo con 50/50: {accuracy_50}')

# Calcular la matriz de confusión para la segunda ejecución
conf_matrix_50 = confusion_matrix(y_test_50, y_pred_50)
print("Matriz de Confusión para la segunda ejecución (50/50):")
print(conf_matrix_50)


Precisión del modelo con 50/50: 0.8840125391849529
Matriz de Confusión para la segunda ejecución (50/50):
[[ 50   1   0   5   6   0]
 [  0  94   0   6   0   0]
 [  0   0   6   2   3   0]
 [  4   4   2 337   8   2]
 [  5   0   1  19  68   0]
 [  0   2   0   4   0   9]]
